In [3]:
import os
import fnmatch
import json

In [4]:
import json
import glob
import os
class RawDataReader:
    def __init__(self, file_path):
        with open(file_path, 'r') as file:
            self.data = json.load(file)

        self.id = self.data['ID']
        self.file_identifier = self.data['patients_file_identifier']
        self.plan_details = self.extract_plan_details()

    def extract_plan_details(self):
        plan_details = []
        for plan in self.data['plans']:
            if plan['completed'] == True:
                plan_name = plan['name']
                planning_exam_name = plan['planning_examination']['name']
                dose_dicom_filename = plan['planning_examination']['dose_DICOM_filename']

                evaluation_exams = []
                for exam in plan['evaluation_examinations']:
                    for opt_case in exam['optimization_cases']:
                        if opt_case.get('completed', True) and 'gradient_descent_result' in opt_case:
                            final_trans_coord = opt_case['gradient_descent_result'].get('final_translation_coordinate', {})
                            x = round(final_trans_coord.get('x', 0), 5)
                            y = round(final_trans_coord.get('y', 0), 5)
                            z = round(final_trans_coord.get('z', 0), 5)
                            exam_detail = {
                                # 'name': exam['name'],
                                'final_translation_coordinate': {'x': x, 'y': y, 'z': z},
                                'dose_dicom_filename': opt_case['gradient_descent_result'].get('dose_DICOM_filename', '')
                            }
                            evaluation_exams.append(exam_detail)

                plan_details.append({
                    'plan_name': plan_name,
                    'planning_exam_name': planning_exam_name,
                    'dose_dicom_filename': dose_dicom_filename,
                    'evaluation_examinations': evaluation_exams
                })
            else:
                plan_details.append({
                    'plan_name': plan_name,
                    'planning_exam_name': None,
                    'dose_dicom_filename': None,
                    'evaluation_examinations': None
                })

        return plan_details

    def get_plan_details(self):
        return self.plan_details
# # Usage
# file_path = '/home/shahpouriz/Data/DBP_newDATA/DBP/results_DBP_OP007_Photons_20231024_053647.json'
# treatment_data = RawDataReader (file_path)
# print(treatment_data.id)
# print(treatment_data.file_identifier)
# print(treatment_data.get_plan_details())


--------------------

In [5]:

class DirectoryProcessor:
    def __init__(self, directory_path):
        self.directory_path = directory_path

    def process_files(self):
        
        # Search for json files matching the pattern
        file_pattern = os.path.join(self.directory_path, 'results_DBP_OP*.json')
        json_files = glob.glob(file_pattern)

        results = []
        for file_path in json_files:
            reader = RawDataReader(file_path)
            file_results = {
                'id': reader.id,
                'file_identifier': reader.file_identifier,
                'plan_details': reader.get_plan_details()
            }
            results.append(file_results)

        return results

# Usage
directory_path = '/home/shahpouriz/Data/DBP_newDATA/DBP'
processor = DirectoryProcessor(directory_path)
all_results = processor.process_files()

# for result in all_results:
#     print(result)

------------
# convert_dicom_to_nrrd

In [8]:
import os
import SimpleITK as sitk

class DICOMtoNRRDConverter:
    def __init__(self, directory_path, output_directory):
        self.directory_path = directory_path
        self.output_directory = output_directory
        self.directory_processor = DirectoryProcessor(directory_path)

    def convert_and_save(self):
        patient_data = self.directory_processor.process_files()

        for patient in patient_data:
            patient_id = patient['id']
            output_path = os.path.join(self.output_directory, patient_id)
            if not os.path.exists(output_path):
                os.makedirs(output_path)

            for plan in patient['plan_details']:
                self._process_dose_file(plan['dose_dicom_filename'], output_path)
                
                for exam in plan['evaluation_examinations']:
                    self._process_dose_file(exam['dose_dicom_filename'], output_path)

    def _process_dose_file(self, dose_dicom_filename, output_path):
        if dose_dicom_filename:
            directory, dose_fn = os.path.split(dose_dicom_filename)
            name, _ = os.path.splitext(dose_fn)
            nrrd_filename = name + '.nrrd'
            nrrd_path = os.path.join(output_path, nrrd_filename)

            if not os.path.isfile(nrrd_path):
                full_dose_path = os.path.join(self.directory_path, dose_dicom_filename)
                dose_img = sitk.ReadImage(full_dose_path)
                sitk.WriteImage(dose_img, nrrd_path)

# Usage
directory_path = '/home/shahpouriz/Data/DBP_newDATA/DBP'
output_directory = '/home/shahpouriz/Data/DBP_newDATA/DBP/nrrd'
converter = DICOMtoNRRDConverter(directory_path, output_directory)
converter.convert_and_save()
